In [1]:
import json
from datasets import load_dataset


In [2]:
from datasets import load_dataset

ds = load_dataset("openai/summarize_from_feedback", "axis")

In [3]:
df_val = ds["validation"].to_pandas()

In [4]:
# unique users
df_val["worker"].drop_duplicates().shape

(32,)

In [5]:
# unique documents
df_val["info"].drop_duplicates().shape

(1038,)

# Preprocess dataset

In [6]:
# preprocess document
df_val =df_val.rename(columns={"worker":"uid"})
df_val["doc_id"] = df_val["info"].apply(lambda x: x["id"])
df_val["doc_text"] = df_val["info"].apply(lambda x: x["post"])
# df_val["site"] = df_val["info"].apply(lambda x: x["site"])
# df_val["subreddit"] = df_val["info"].apply(lambda x: x["subreddit"])

df_val["summary_text"] = df_val["summary"].apply(lambda x: x["text"])
df_val["summary_rating"] = df_val["summary"].apply(lambda x: x["axes"]["overall"])
df_val["summary_model"] = df_val["summary"].apply(lambda x: x["policy"])
df_val = df_val[["doc_id", "doc_text", "uid", "summary_model", "summary_text", "summary_rating"]] 
df_val

,doc_id,doc_text,uid,summary_model,summary_text,summary_rating
0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_ppo_rm4_t.7,Fiance and I recently got infected with scabi...,5.0
1,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,pretrain_6b_t.7,my fiance and I refuse to treat our room mate...,4.0
2,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_6b_ppo_rm4_6b_t.7,"fiancé and I contracted scabies, roommate ref...",6.0
3,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_6b_t0.7,"fiance and I are infected with scabies, room ...",6.0
4,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_12b_t0.7,"Fiance and I contracted scabies, roommate ref...",6.0
...,...,...,...,...,...,...
8580,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,sup4_12b,"I want to ask out a girl on a date, general t...",6.0
8581,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,ref,"Girl ignored me again, I cease conversation. ...",1.0
8582,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,title,[Update 2] I [18 M] want to ask out a girl [1...,7.0
8583,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,lead2,"[Original](\n(Clarification on this one, I di...",1.0


# appropriate ratings data to create user summaries

In [8]:
# pick user max ratings as their own summary
# Assumption: For any document, the highest rated summary by the user can be taken as the summary generated by that user
idx = df_val.groupby(['doc_id', 'uid'])['summary_rating'].idxmax()
df_val.loc[idx].to_csv("./openai_reddit_appropriated_user_summary.csv", index=False)


In [9]:
df_val

,doc_id,doc_text,uid,summary_model,summary_text,summary_rating
0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_ppo_rm4_t.7,Fiance and I recently got infected with scabi...,5.0
1,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,pretrain_6b_t.7,my fiance and I refuse to treat our room mate...,4.0
2,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_6b_ppo_rm4_6b_t.7,"fiancé and I contracted scabies, roommate ref...",6.0
3,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_6b_t0.7,"fiance and I are infected with scabies, room ...",6.0
4,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",iL7GfrbN2PeB3KInidqSxUdxYcTZmG,sup4_12b_t0.7,"Fiance and I contracted scabies, roommate ref...",6.0
...,...,...,...,...,...,...
8580,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,sup4_12b,"I want to ask out a girl on a date, general t...",6.0
8581,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,ref,"Girl ignored me again, I cease conversation. ...",1.0
8582,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,title,[Update 2] I [18 M] want to ask out a girl [1...,7.0
8583,t3_3i230d,"[Original](\n(Clarification on this one, I did...",uvzut5OK2bvei9zoCDdktcfLENYioY,lead2,"[Original](\n(Clarification on this one, I di...",1.0
